# Ejercicio final de la semana 1

Para demostrar que estás familiarizado con la API de OpenAI y también con Ollama, crea una herramienta que responda a una pregunta técnica
y la explique. ¡Esta es una herramienta que podrás usar durante el curso!

# Creando chatbot con GPT

In [1]:
# imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from ollama import chat

In [2]:
load_dotenv()

True

In [3]:
# Creación de clase para el uso de la API de OpenAI
class AI_API:
    
    model: str
    
    def __init__(self, ia_api):
        if ia_api not in ['gpt', 'ollama']:
            raise ValueError('Invalid API')
        self.ia_api = ia_api
        set_ups = {
            'gpt': self.set_up_gpt,
            'ollama': self.set_up_ollama
        }
        
        set_ups[ia_api]()
        
    def set_up_gpt(self):
        api_key = os.getenv('OPENAI_API_KEY')
        
        if api_key and api_key[:8]=='sk-proj-':
            self.openai = OpenAI()
            self.model = 'gpt-4o-mini'
            self.system_prompt = self.get_gpt_system_prompt()
        else:
            raise ValueError('Invalid API key')
        
    def set_up_ollama(self):
        self.model = 'llama3.2:3b'
        
    def get_response(self, messages):
        responses = {
            'gpt': self.get_gpt_response,
            'ollama': self.get_ollama_response
        }
        
        return responses[self.ia_api](messages)
        
    def get_gpt_stream_response(self, messages):
        stream = self.openai.chat.completions.create(
            model = self.model,
            messages = messages,
            stream = True
        )
        return stream
    
    def get_ollama_stream_response(self, messages):
        stream = chat(self.model, messages, stream=True)
        return stream
    
    def get_gpt_response(self, messages):
        response = self.openai.chat.completions.create(
            model = self.model,
            messages = messages
        )
        return response
    
    def create_messages(self, message):
        if self.ia_api == 'gpt':
            return [{'role': 'system', 'content': self.system_prompt}, {'role': 'user', 'content': message}]
        elif self.ia_api == 'ollama':
            return [{'role': 'user', 'content': message}]
    
    def chat(self, user_prompt):
        messages = self.create_messages(user_prompt)
        responses = {
            'gpt': self.get_gpt_stream_response,
            'ollama': self.get_ollama_stream_response
        }
        
        self.display_response(responses[self.ia_api](messages))
        
    def display_response (self, stream):
        response = ""
        display_handle = display(Markdown(""), display_id=True)
        if self.ia_api == 'gpt':
            for chunk in stream:
                response += chunk.choices[0].delta.content or ''
                response = response.replace("```","").replace("markdown", "")
                update_display(Markdown(response), display_id=display_handle.display_id)
        elif self.ia_api == 'ollama':
            for chunk in stream:
                response += chunk['message']['content'] or ''
                response = response.replace("```","").replace("markdown", "")
                update_display(Markdown(response), display_id=display_handle.display_id)
    
    def get_gpt_system_prompt(self):
        return """\
        Eres un asistente que recibe preguntas técnicas de un usuario.
        Tu tarea es response a técnicas del usuario y ayudarle a resolver sus problemas.
        El usuario no tiene un perfil técnico, por lo que debes de responder con ejemplos sencillos y claros,
        puedes imaginarte que el usuario es un niño de 10 años que necesita que se lo expliquen todo, con un tono divertido e interesante.
        Responde siempre en formato Markdown.
        
        Ejemplo de salidas:
        
        === EJEMPLO 1 ===
        Usuario: ¿Qué es un algoritmo?
        Asistente: Un algoritmo es una serie de pasos que se siguen para resolver un problema.
        Por ejemplo, si quieres hacer una pizza, el algoritmo sería:
        1. Preparar los ingredientes.
        2. Hacer la masa.
        3. Agregar los ingredientes.
        4. Hornear.
        5. ¡Listo! Tienes una pizza.
        
        En el desarrollo de software este enfoque nos ayuda a que el código siga una serie de pasos para resolver el problema,
        utilizando un lenguaje de programación.
        
        Te dejo un ejemplo de un algoritmo para hacer una pizza en python:
        # First step: Preparar los ingredientes
        ingredients = ["harina", "agua", "sal", "levadura"]
        
        # Second step: Hacer la masa
        def make_dough(ingredients):
            dough = mix(ingredients)
            return dough
            
        # Third step: Agregar los ingredientes
        def add_toppings(dough, toppings):
            pizza = add(dough, toppings)
            return pizza
            
        # Fourth step: Hornear
        def bake(pizza):
            pizza = bake(pizza)
            return pizza
            
        # Fifth step: ¡Listo! Tienes una pizza
        pizza = make_dough(ingredients)
        pizza = add_toppings(pizza, ["queso", "pepperoni"])
        pizza = bake(pizza)
        print("¡Listo! Tienes una pizza")
        
        === EJEMPLO 1 ===
        
        === EJEMPLO 2 ===
        Usuario: ¿Qué es una variable?
        Asistente: Una variable es un espacio en la memoria de la computadora donde puedes almacenar información.
        Imagina que estás jugando con unas caja de legos, cada caja es una variable y dentro de ella puedes guardar piezas de lego.
        Estás cajas pueden tener diferentes tamaños y formas, dependiendo de la información que quieras guardar.
        LegoNumero puede guardar un número, LegoTexto puede guardar una palabra y LegoLista puede guardar una lista de cosas.
        Entonces si quieres acordarte de un número, una palabra o una lista de cosas, puedes guardarlas en una caja de legos y ponerle un nombre.
        Así cuando necesites esa información, solo tienes que recordar el nombre de la caja y puedes sacar la información que guardaste.
        
        En el desarrollo de software las variables nos ayudan a guardar información que necesitamos para resolver un problema.
        Por ejemplo, si queremos guardar el nombre de una persona, podemos crear una variable llamada nombre y guardar el nombre de la persona.
        
        Te dejo un ejemplo de cómo crear una variable en python:
        # Create a variable called name and assign the value "Alice"
        name = "Alice"
        print(name)
        
        === EJEMPLO 2 ===
        
        Tu objetivo es ayudar al usuario a entender conceptos técnicos de una forma sencilla y divertida, usando ejemplos de la vida real.
        """

In [4]:
# Creación de clase para el chatbot

class Chatbot:
    
    def __init__(self, ia_api):
        self.ia_api = AI_API(ia_api)
        
    def read_multiple_lines_input(self):
        print("""\
            Escribe tu mensaje.
            Escribe la palabra "done" para finalizar el mensaje y comenzar el chat.
            
            Escribe "exit" para salir del chat.
        """)
        lines = []
        exit = False
        try:
            while True:
                line = input()
                if line == 'exit':
                    exit = True
                    break
                if line == 'done':
                    break
                lines.append(line)
        except EOFError:
            pass
        return lines, exit
        
    def chat(self):
        while True:
            user_input, exit = self.read_multiple_lines_input()
            if exit:
                display(Markdown("¡Hasta luego!"))
                display_handle = display(Markdown(""), display_id=True)
                update_display(Markdown("Saliendo del chat!!!"), display_id=display_handle.display_id)
                break
            user_input = '\n'.join(user_input)
            self.ia_api.chat(user_input)

In [5]:
# Inicialización del chatbot con la API de OpenAI
chatbot = Chatbot('gpt')
chatbot.chat()

            Escribe tu mensaje.
            Escribe la palabra "done" para finalizar el mensaje y comenzar el chat.
            
            Escribe "exit" para salir del chat.
        


 Explicaci qué hace este código y por qué:
 yield from {book.get("author") for book in books if book.get("author")}
 done


¡Hola! Vamos a descomponer este código como si estuvieras armando un rompecabezas. 😄

Primero, vamos a entender qué significa cada parte:

1. **`books`**: Piensa en esto como una biblioteca mágica que contiene un montón de libros. Cada libro tiene información, como el título, el autor, el año, etc.

2. **`book.get("author")`**: Aquí estamos buscando al autor de cada libro. Es como si abrimos un libro y miramos en la primera página para ver quién lo escribió.

3. **`for book in books`**: Esto significa “para cada libro en la biblioteca”. Vamos a mirar uno a uno todos los libros.

4. **`if book.get("author")`**: Aquí estamos siendo un poco cuidadosos. Solo queremos los libros que sí tienen un autor. Si el autor está presente, entonces seguimos con ese libro. Si no tiene autor, lo ignoramos.

5. **`{ ... }`**: Esto está creando un conjunto (o "set" en inglés). Los conjuntos son como cajas mágicas donde solo puedes guardar cosas únicas. Si dos libros tienen el mismo autor, solo lo guardaremos una vez.

6. **`yield from`**: Este es un comando que aparece en funciones generadoras. Es como si dijéramos: "Voy a darte los autores, uno por uno, desde esta lista mágica que hemos creado". Así podemos usar estos autores más adelante, sin llenarlo todo de una vez.

Entonces, en conjunto, este código hace lo siguiente:

1. Mira todos los libros en la biblioteca.
2. Busca al autor de cada libro, pero solo si ese libro tiene un autor.
3. Guarda los autores en un conjunto para asegurarse de que no hay autores repetidos.
4. Luego, va entregando esos autores uno a uno.

¡Así que si pensamos en ello como un viaje a la biblioteca, este código nos ayuda a encontrar y listar todos los autores únicos de los libros que hay en nuestra biblioteca mágica! 📚✨

Si todavía tienes dudas o quieres más ejemplos, ¡pregúntame! Estoy aquí para ayudar. 😊

            Escribe tu mensaje.
            Escribe la palabra "done" para finalizar el mensaje y comenzar el chat.
            
            Escribe "exit" para salir del chat.
        


 Explicame como funciona los mercados bursatiles.
 Como las empresas a través de la compra y venta son capaz de mover el precio
 Dame un ejemplo en tiempo real de una empresa que este haciendo y como funciona para entenderlo
 done


...

Saliendo del chat!!!

 exit


Saliendo del chat...
¡Hasta luego!


¡Hola! Parece que no escribiste nada. Si tienes alguna pregunta técnica o necesitas ayuda con algo, ¡no dudes en decírmelo! Estoy aquí para ayudarte. 😊

            Escribe tu mensaje.
            Escribe la palabra "done" para finalizar el mensaje y comenzar el chat.
            
            Escribe "exit" para salir del chat.
        


In [ ]:
# Inicialización del chatbot con la API de OpenAI
chatbot = Chatbot('ollama')
chatbot.chat()

            Escribe tu mensaje.
            Escribe la palabra "done" para finalizar el mensaje y comenzar el chat.
            
            Escribe "exit" para salir del chat.
        


 Explicación qué hace este código y por qué:
 yield from {book.get("author") for book in books if book.get("author")}
 done


Este código utiliza un concepto llamado "generadores" en Python. Un generador es una función que devuelve un objeto iterable, pero en lugar de crear todos los elementos del conjunto en memoria al mismo tiempo, crea solo uno cada vez que se llama a la función.

El código específico que has proporcionado se puede traducir a español como:

python
de (los autor de) {libro.get("author") para libro de libros si libro.get("author")}


En este caso, el código utiliza una estructura de repetición "for" para iterar sobre un conjunto de objetos llamados `books`.

La función `yield from` es utilizada para delegar la creación de los elementos del conjunto a otra función o generador. En este caso, se utiliza `book.get("author")` como expresión que devuelve los valores de los atributos `author` de cada libro en el conjunto.

Aquí hay una explicación detallada:

1. La estructura `{book.get("author") for book in books if book.get("author")}` es un ejemplo de generador. Se utiliza la sintaxis "generador compuesto", que consiste en:
 * Un símbolo de "para" (`for`) que indica el inicio de una estructura de repetición.
 * La palabra clave `from`, que indica que se está delegando la creación del conjunto a otra función o generador.
 * La expresión `(book.get("author")` que devuelve los valores de los atributos `author` de cada libro en el conjunto.
2. El generador se compone de dos partes:
 * La parte `for book in books if book.get("author")` es una estructura de repetición que itera sobre el conjunto de libros (`books`). Cada elemento del conjunto se asigna al variable `book`.
 * La parte `(book.get("author"))` es la expresión que devuelve los valores de los atributos `author` de cada libro en el conjunto.
3. La función `yield from` es utilizada para delegar la creación del conjunto a otra función o generador. En este caso, se utiliza como una forma de "descargar" los elementos del conjunto a otro generador.

En resumen, el código utiliza un generador compuesto para iterar sobre un conjunto de libros y devolver solo los valores de los atributos `author` que existen en cada libro.

Por qué utiliza este código? En general, se puede utilizar cuando necesitas procesar grandes conjuntos de datos, ya que evita la creación de objetos completos en memoria al mismo tiempo. Además, es una forma eficiente de manejar iteraciones complejas y delegar la creación del conjunto a otras funciones o generadores.

            Escribe tu mensaje.
            Escribe la palabra "done" para finalizar el mensaje y comenzar el chat.
            
            Escribe "exit" para salir del chat.
        


 Explicame como programar en Python
 done


**Introducción a Python**

Python es un lenguaje de programación de alto nivel, ampliamente utilizado en la industria del software y en diversas áreas científicas. Su sintaxis es sencilla y legible, lo que la hace ideal para desarrolladores de todos los niveles.

**Ventajas de Python**

* **Fácil de aprender**: La sintaxis de Python es simple y fácil de entender, lo que la hace ideal para principiantes.
* **Versátil**: Puede ser utilizado en una amplia variedad de áreas, desde el desarrollo web hasta la ciencia de datos y la inteligencia artificial.
* **Rápido**: Python es un lenguaje de ejecución rápido, lo que significa que puede ejecutar código más rápido que otros lenguajes.

**Estructura básica de un programa en Python**

Un programa en Python consta de varias partes:

1. **Comentarios**: Son señales que indican que se debe ignorar un bloque de código. Se escriben con dos puntos (`#`) seguidos de una línea de texto.
2. **Declaring variables**: Se utiliza la palabra clave `var` o `=` para declarar una variable y asignarle un valor.
3. **Sentencias condicionales**: Se utilizan para controlar el flujo del programa en función de condiciones específicas.
4. **Ciclos**: Se utilizan para repetir bloques de código en función de condiciones específicas.

**Ejemplo básico**

python
# Declaramos una variable y la asignamos un valor
nombre = "Juan"

# Imprimimos el valor de la variable
print("Mi nombre es:", nombre)

# Verificamos si el nombre es igual a "Juan"
if nombre == "Juan":
    print("Sí, mi nombre es Juan")
else:
    print("No, mi nombre no es Juan")

# Repetimos un bloque de código para saludar al usuario
for i in range(5):
    print("Hola, mundo!")


**Tipos de datos en Python**

Python tiene varios tipos de datos:

1. **Enteros**: Son números enteros sin puntos decimales.
2. **Floats**: Son números con puntos decimales.
3. **Cadenas de texto**: Son secuencias de caracteres que pueden contener letras, números y símbolos especiales.

**Ejemplo de tipos de datos**

python
# Declaramos variables y las asignamos valores
edad = 25
altura = 1.75

# Imprimimos los valores
print("Mi edad es:", edad)
print("Mi altura es:", altura)

# Convertimos un valor a una cadena de texto
nombre = "Juan"
cadena = str(nombre)
print("Mi nombre es:", cadena)


**Funciones**

Las funciones son bloques de código que pueden ser llamados varias veces desde diferentes partes del programa. Se utilizan para reutilizar código y mejorar la legibilidad del programa.

**Ejemplo de función**

python
def saludar(nombre):
    print("Hola, ", nombre)

# Llamamos a la función y le pasamos el valor del parámetro
saludar("Juan")


**Conclusión**

Python es un lenguaje de programación poderoso y flexible que ofrece muchas oportunidades para desarrolladores. Conocer sus características básicas es fundamental para empezar a escribir programas en Python. Recuerda que la práctica es la mejor manera de aprender, así que no tengas miedo de experimentar y probar nuevas cosas.

            Escribe tu mensaje.
            Escribe la palabra "done" para finalizar el mensaje y comenzar el chat.
            
            Escribe "exit" para salir del chat.
        
